## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.tail()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
570,570,Bundaberg,AU,-24.8500,152.3500,52.21,89,0,4.00,clear sky
571,571,Palana,RU,59.1167,159.9667,56.46,67,95,7.07,overcast clouds
572,572,Lac Du Bonnet,CA,50.2536,-96.0612,74.89,40,0,12.35,clear sky
573,573,Ust-Maya,RU,60.4167,134.5333,53.80,95,100,2.80,light rain
574,574,Gillette,US,44.2911,-105.5022,95.04,18,0,3.44,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input('What is the minimum temperature you would like for your trip? '))

max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &(city_data_df['Max Temp'] >= min_temp)]


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID        195
City           195
Country        195
Lat            195
Lng            195
Max Temp       195
Humidity       195
Cloudiness     195
Wind Speed     195
Description    195
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,81.18,scattered clouds,-0.6000,73.0833,
1,Balabac,PH,80.35,broken clouds,7.9868,117.0645,
2,Yarmouth,US,83.97,scattered clouds,41.7057,-70.2286,
5,Albany,US,85.64,broken clouds,42.6001,-73.9662,
6,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,
11,Coari,BR,87.93,broken clouds,-4.0850,-63.1414,
19,Hasaki,JP,81.28,overcast clouds,35.7333,140.8333,
23,Salalah,OM,77.09,mist,17.0151,54.0924,
24,San Cristobal,VE,82.58,broken clouds,7.7669,-72.2250,
29,Cayenne,GF,86.04,clear sky,4.9333,-52.3333,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        
        print('Hotel not found... skipping')


Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [24]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna()

clean_hotel_df.tail()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
564,Chongqing,CN,89.92,clear sky,29.5628,106.5528,Somerset Jiefangbei Chongqing Hotel
566,Tripoli,LY,82.00,clear sky,32.8752,13.1875,Al Waddan Hotel
567,Benton Harbor,US,77.65,clear sky,42.1167,-86.4542,The Boulevard Inn & Bistro
568,Bargi,IN,76.28,broken clouds,22.9833,79.8667,Aditya Dubey
569,Alexander City,US,89.06,clear sky,32.9440,-85.9539,Hampton Inn Alexander City


In [25]:
# 8a. Create the output File (CSV)
output_data_file = ('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>


"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 

# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))